In [2]:
from typing import Optional, List, Dict, Generator, Any, Tuple
from collections import defaultdict, deque


import requests
from pydantic import HttpUrl

from srl_notion.api_connection import NotionAPIConnector
from srl_notion.api_result_fetcher import ResultFetcher
from srl_notion.dict_deque import DictQueueStructure
from srl_notion.updater import DictDequeueStructureUpdater

connector = NotionAPIConnector()
raw_response:dict = connector.main()
headers = connector.headers

fetcher = ResultFetcher(raw_response=raw_response, headers=headers)

res = fetcher.main()

In [3]:
res

defaultdict(collections.deque,
            {'Ton per': deque([{'OMG une page ! (1)': 'https://www.notion.so/OMG-une-page-1-14c3cb01ecb04518a3641291e6e4d109'},
                    {'Truc (1)': 'https://www.notion.so/Truc-1-604d0ff2fe544013bb6708b58f8ba65c'}]),
             'ta mere': deque([{'ta mere': 'https://www.notion.so/ta-mere-80497c9312324710b14db45a27563932'}]),
             'Page test': deque([{'Propal Raphael ': 'https://www.notion.so/Propal-Raphael-e266eee4b5134c48a7937dfa05b5240f'},
                    {'ma bite t': 'https://www.notion.so/ma-bite-t-7c547acf50de431a8b9559858a152330'},
                    {'Machin': 'https://www.notion.so/Machin-ee95162e13c349f38ea78253611910d9'}])})

In [6]:
import pickle
import pathlib

def load_pickle(path_object: pathlib.Path) -> Any:
    with open(path_object, "rb", encoding="utf-8") as f:  # rb -> Read Binary
        return pickle.load(f)

@staticmethod
def save_pickle(object_: Any, path_to_save: pathlib.Path) -> None:
    with open(path_to_save, "wb", encoding="utf-8") as f:
        pickle.dump(object_, f)

PosixPath('/home/lucas/all_code/notion_srl/space-repetition-learning-with-notion/data_structure.pkl')

In [ ]:
updater = DictDequeueStructureUpdater(response_result_fetcher=res)
updater

In [ ]:
class InterfaceUser:
    
    """ 
    Bridge between the user and the data structure
    Take the up to date data structure, modify it as the user interact with it 
    """
    
class DictDequeueStructureUpdater:
    
    """ 
    Take the raw response from the result fetcher, and the previous saved data structure
    update the data structure before going to the inferface
    Save the structure with it's new order at the end of the execution
    """